In [3]:
import numpy as np
import pandas as pd
import import_ipynb
import pair_selection_DBSCAN_funclist_for_training as myFunc
import matplotlib.pyplot as plt

import math
import sklearn
import datetime

from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')

In [4]:
price_df = pd.read_csv('./Data/us_etf_price.csv')
etf_info = pd.read_csv('./Data/etfs_details_equity.csv')
er_df = etf_info[['Symbol','ER']]
er_df = er_df.set_index('Symbol')
er_df = er_df['ER'].apply(lambda x: x.split('%')[0]).astype(float)

In [5]:
pc_selecting_threshold = 0.9
eps = 1.8
min_samples = 2
cluster_size_limit = 100
cluster_member_counts = 100
inverse_threshold=-0.99
coint_pvalue_threshold=0.01
hurst_threshold=0.5
half_life_threshold=10
mean_reverting_freq=12

In [6]:
def get_pair(start_date, end_date):

    _, close_df, rtn_df, low_volume_etf = myFunc.preprocessing(price_df,etf_info, start_date, end_date)
    pc_rtn = myFunc.get_pca_return(rtn_df, pc_selecting_threshold)
    clusters_viz_list, clustered_series = myFunc.dbscan_clustering(close_df, pc_rtn, eps, min_samples, cluster_size_limit, cluster_member_counts)
    selected_pair, short_pair = myFunc.Pair_selection(close_df=close_df,
                                                rtn_df=rtn_df,
                                                low_volume_etf=low_volume_etf,
                                                clusters_viz_list=clusters_viz_list, 
                                                clustered_series=clustered_series, 
                                                inverse_threshold=inverse_threshold, 
                                                coint_pvalue_threshold=coint_pvalue_threshold, 
                                                hurst_threshold=hurst_threshold, 
                                                half_life_threshold=half_life_threshold, 
                                                mean_reverting_freq=mean_reverting_freq)

    return selected_pair, short_pair, close_df, rtn_df

In [7]:
monthly = pd.date_range('2009-12-31', '2022-1-1', freq='MS')
period_candi = [(6,1), (12,1), (3,1)]

In [8]:
# 포메이션 - 트레이딩 기간 조합별 period 뽑기
formation_train = {}
for cnd in period_candi:

    train_window_width = cnd[0]
    test_window_width = cnd[1]

    train_period = []
    test_period = []
    for i in range(len(monthly)- train_window_width-test_window_width):
        train_start_date = monthly[i]
        train_end_date = monthly[i+train_window_width]
        test_end_date = monthly[i+train_window_width+test_window_width]

        if test_end_date.year < 2017:

            train_period.append((train_start_date.strftime("%Y-%m-%d"), train_end_date.strftime("%Y-%m-%d"), test_end_date.strftime("%Y-%m-%d")))
        
        elif test_end_date.year >=2017:

            test_period.append((train_start_date.strftime("%Y-%m-%d"), train_end_date.strftime("%Y-%m-%d"), test_end_date.strftime("%Y-%m-%d")))
            
    formation_train[cnd] = (train_period, test_period)

In [9]:
# 트레이딩 기간이 2021년인 거 뽑음
performance_test = formation_train[(12,1)][1][49:]

In [10]:
result = {}
for i in range(len(performance_test)):

    try:
        print('=================== {} 번째 시작 ======================'.format(i))
        start_date = performance_test[i][0]
        end_date = performance_test[i][1]

        selected_pair, short_pair, close_df, rtn_df = get_pair(start_date, end_date)

    except KeyboardInterrupt as e:
        print(e)
        pass

    result[performance_test[i]] = (selected_pair, short_pair, close_df, rtn_df)

=================== 0 번째 시작 ======================


100%|██████████| 1338/1338 [00:20<00:00, 64.94it/s]


Clusters discovered: 10
Clusters formed: 9
Pairs to evaluate: 1374
final_clusters index :  [8, 7, 9, 4, 6, 5, 3, 1, 2]


100%|██████████| 1/1 [00:00<00:00, 179.95it/s]
0it [00:00, ?it/s]/9 [00:02<00:22,  2.87s/it]
0it [00:00, ?it/s]/9 [00:04<00:14,  2.02s/it]
100%|██████████| 3/3 [00:00<00:00, 241.80it/s]
0it [00:00, ?it/s]/9 [00:20<00:12,  4.14s/it]
0it [00:00, ?it/s]/9 [00:22<00:07,  3.72s/it]
100%|██████████| 9/9 [01:09<00:00,  7.69s/it]


=================== 1 번째 시작 ======================


...

0it [00:00, ?it/s]/109 [00:00<?, ?it/s]
0it [00:00, ?it/s]/109 [00:01<03:15,  1.81s/it]
0it [00:00, ?it/s]/109 [00:03<03:13,  1.81s/it]
0it [00:00, ?it/s]/109 [00:04<02:27,  1.39s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]/109 [00:06<01:55,  1.11s/it]
0it [00:00, ?it/s]/109 [00:07<01:48,  1.05s/it]
100%|██████████| 1/1 [00:00<00:00, 154.42it/s]]
0it [00:00, ?it/s]/109 [00:11<02:59,  1.78s/it]
0it [00:00, ?it/s]/109 [00:12<02:31,  1.52s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]1/109 [00:14<02:01,  1.24s/it]
100%|██████████| 1/1 [00:00<00:00, 172.05it/s]t]
0it [00:00, ?it/s]3/109 [00:19<02:53,  1.81s/it]
0it [00:00, ?it/s]4/109 [00:20<02:52,  1.81s/it]
0it [00:00, ?it/s]5/109 [00:21<02:26,  1.56s/it]
0it [00:00, ?it/s]6/109 [00:22<02:07,  1.37s/it]
0it [00:00, ?it/s]7/109 [00:23<01:53,  1.23s/it]
0it [00:00, ?it/s]8/109 [00:25<02:06,  1.39s/it]
0it [00:00, ?it/s]9/109 [00:27<02:15,  1.51s/it]
0it [00:00, ?it/s]0/109 [00:27<01:58,  1.33s/it]
0it [00:00, ?it/s]1/109 [00:28<01:45,  1.20s/it]

=================== 11 번째 시작 ======================


100%|██████████| 1489/1489 [00:29<00:00, 51.18it/s]


Clusters discovered: 100
Clusters formed: 100
Pairs to evaluate: 17396
final_clusters index :  [99, 63, 62, 61, 60, 59, 58, 55, 54, 52, 48, 47, 45, 43, 42, 36, 33, 31, 30, 29, 27, 25, 16, 14, 13, 12, 10, 7, 65, 68, 50, 70, 83, 98, 82, 86, 69, 87, 88, 89, 90, 91, 92, 76, 75, 73, 72, 93, 94, 95, 96, 85, 39, 26, 21, 18, 24, 8, 66, 57, 84, 79, 74, 53, 71, 1, 77, 67, 46, 23, 64, 81, 19, 44, 97, 4, 41, 5, 56, 37, 22, 78, 17, 80, 15, 51, 32, 35, 28, 40, 20, 38, 0, 6, 49, 34, 11, 9, 2, 3]


0it [00:00, ?it/s]/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]/100 [00:01<03:01,  1.84s/it]
0it [00:00, ?it/s]/100 [00:02<02:07,  1.30s/it]
0it [00:00, ?it/s]/100 [00:04<02:29,  1.54s/it]
0it [00:00, ?it/s]/100 [00:05<02:04,  1.30s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]/100 [00:07<01:43,  1.10s/it]
0it [00:00, ?it/s]/100 [00:08<01:38,  1.06s/it]
0it [00:00, ?it/s]/100 [00:09<01:34,  1.03s/it]
0it [00:00, ?it/s]/100 [00:11<01:57,  1.29s/it]
0it [00:00, ?it/s]0/100 [00:13<02:11,  1.46s/it]
0it [00:00, ?it/s]1/100 [00:14<01:56,  1.30s/it]
0it [00:00, ?it/s]2/100 [00:14<01:44,  1.19s/it]
0it [00:00, ?it/s]3/100 [00:16<01:59,  1.38s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]5/100 [00:17<01:21,  1.05it/s]
0it [00:00, ?it/s]6/100 [00:18<01:19,  1.06it/s]
0it [00:00, ?it/s]7/100 [00:19<01:17,  1.07it/s]
0it [00:00, ?it/s]8/100 [00:20<01:16,  1.07it/s]
0it [00:00, ?it/s]9/100 [00:23<01:57,  1.45s/it]
0it [00:00, ?it/s]0/100 [00:25<02:04,  1.56s/it]
0it [00:00, ?it/s]1/100 [00:26<02:09,  1.64s/it]

In [19]:
result

{('2020-01-01',
  '2021-01-01',
  '2021-02-01'): ([[],
   [],
   [],
   [],
   [],
   [],
   [],
   [],
   [('SPXS',
     'SDOW')]], {'SPXS': 'SPXL', 'DOG': 'DIA', 'PSQ': 'QLD', 'QID': 'QLD', 'SPDN': 'SPXL', 'SRTY': 'TNA', 'DXD': 'DIA', 'TECS': 'XLK', 'SH': 'VOO', 'SPXU': 'SPXL', 'SMDD': 'IJH', 'TWM': 'UWM', 'SQQQ': 'TQQQ', 'SSG': 'USD', 'SDOW': 'DIA', 'SOXS': 'SOXX', 'TZA': 'URTY', 'SEF': 'UYG', 'SDS': 'UPRO', 'SKF': 'UYG', 'MYY': 'IJH', 'RWM': 'URTY', 'SMN': 'UYM', 'LABD': 'XBI', 'FAZ': 'FAS'},                    IVE        SIL       ROBO       FINX        FKU  \
  Date                                                                 
  2020-01-02  130.589996  32.939999  42.610001  30.840000  41.259998   
  2020-01-03  129.660004  32.689999  42.099998  30.660000  40.610001   
  2020-01-06  129.869995  32.320000  42.169998  30.660000  40.529999   
  2020-01-07  129.380005  32.599998  42.209999  30.629999  40.470001   
  2020-01-08  129.720001  31.389999  42.450001  30.850000  40.419998